# Training Bengali News Word Vectors

In this notebook, we will use the data we scraped from news websites to train a Word2Vec model for Bengali.

Then we will test the model to see how well it is performing.

First we import the packages we need

In [1]:
import json
import os
import re
import string
import numpy as np

from gensim.models import Word2Vec

Let's define a function that will read the data file and extract the fields we want.

In our case, we will be using the article body for training

In [2]:
def extract_text(filename, field):
    
    extracted_field=[]
    
    with open(os.path.join('data', filename), 'r') as f:
        articles=json.load(f)
    
    for article in articles['articles']:
        extracted_field.append(article[field].strip())
    
    return extracted_field

Now we define a function to preprocess our data.

The function does the following:
- It replaces common texts found in the data and replaces that with our custom text
- It removes all emoji's and emoticons from the text
- It removes all English text

In [3]:
def replace_strings(texts, replace):
    new_texts=[]
    
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    english_pattern=re.compile('[a-zA-Z0-9]+', flags=re.I)
    
    for text in texts:
        for r in replace:
            text=text.replace(r[0], r[1])
        text=emoji_pattern.sub(r'', text)
        text=english_pattern.sub(r'', text)
        text=re.sub(r'\s+', ' ', text).strip()
        new_texts.append(text)

    return new_texts

We also need to remove all the punctuations in our data. The `remove_pun` function removes all common punctuations found in text.

In [4]:
def remove_punc(sentences):
    # import ipdb; ipdb.set_trace()
    new_sentences=[]
    exclude = list(set(string.punctuation))
    exclude.extend(["’", "‘", "—"])
    for sentence in sentences:
        s = ''.join(ch for ch in sentence if ch not in exclude)
        new_sentences.append(s)
    
    return new_sentences

Let's extract some of the data from Ebala and print them to see how the data changes throughout the process.

In [5]:
ebala_body=extract_text('ebala_articles.txt', 'body')

print("\x1b[31mCrawled Unprocessed Text\x1b[0m")
print(ebala_body[12])

replace=[('\u200c', ' '),
         ('\u200d', ' '),
        ('\xa0', ' '),
        ('\n', ' '),
        ('\r', ' ')]

ebala_body=remove_punc(ebala_body)

print("\x1b[31mSentences after removing all punctuations\x1b[0m")
print(ebala_body[12])

ebala_body=replace_strings(ebala_body, replace)

print("\x1b[31mSentences after replacing strings\x1b[0m")
print(ebala_body[12])

Crawled Unprocessed Text
সানিয়া মির্জা কি ইতিমধ্যেই সন্তানের জন্ম দিয়েছেন? হঠাৎই এমন গুজবে উত্তাল সোশ্যাল মিডিয়া। শেষ পর্যন্ত সেই গুজব খণ্ডনে আসরে নামতে হল শোয়েব মালিককে। তিনি টুইট করলেন, ‘‘আমরা সঠিকভাবে সকলকে জানাব যখন আমাদের সন্তান ভূমিষ্ঠ হবে। অনুগ্রহ করে আমাদের জন্য প্রার্থনা করবেন। প্লিজ ইন্টারনেটে যা দেখবেন/পড়বেন তা মোটেই বিশ্বাস করবেন না।’’

We will do a proper announcement when the kid decides to arrive, please keep us in your Duas (and don’t believe everything you see/read on the internet please!) 🙏🏼
— Shoaib Malik 🇵🇰 (@realshoaibmalik) October 14, 2018

ঘটনা হল, কিছুদিনের মধ্যেই শোয়েব-সানিয়ার নতুন সন্তান আসতে চলেছে। আসন্নপ্রসবা সানিয়াকে গত কয়েকমাস ধরেই নিজের সোশ্যাল মিডিয়া অ্যাকাউন্ট থেকে একাধিক ছবি পোস্ট করতে দেখা যাচ্ছে। বিভিন্ন অনুষ্ঠানে পরিবারের সদস্যদের সঙ্গেই সানিয়াকে দেখা গিয়েছে সেই সব পোস্ট করা ছবিতে।

এই বিষয়ে অন্যান্য খবর

সন্তানসম্ভবা সানিয়াকে ‘ইট’ নেটিজেনদের, পালটা ‘পাটকেল’ টেনিস রানির


এর মধ্যেই হঠাৎ চলতি সপ্তাহে গুজব ছড়িয়ে পড়ে সানিয়া ইতিমধ্যেই

We do the same thing for the other data too

In [6]:
abz_body=extract_text('anandabazar_articles.txt', 'body')

abz_body=remove_punc(abz_body)
abz_body=replace_strings(abz_body, replace)

In [7]:
zee_body=extract_text('zeenews_articles.txt', 'body')

zee_body=remove_punc(zee_body)
zee_body=replace_strings(zee_body, replace)

In [8]:
body=[]
body.extend(zee_body)
body.extend(abz_body)
body.extend(ebala_body)

print(f"Total Number of training data: {len(body)}")

Total Number of training data: 14205


Finally, we need to split the articles into sentences and extract each word from those sentences.

Our final training data looks like this

In [9]:
body=[article.split('।') for article in body]
body=[item for sublist in body for item in sublist]
body=[item.strip() for item in body if len(item.split())>1]

body=[item.split() for item in body]

print(body[:10])

[['যা', 'আমাদের', 'ত্যাগের', 'দিকে', 'তপস্যার', 'দিকে', 'নিয়ে', 'যায়', 'তাকেই', 'বলি', 'মনুষ্যত্ব', 'মানুষের', 'ধর্ম'], ['এইরকমই', 'এক', 'ধর্মের', 'কথা', 'ভেবেছিলেন', 'রবীন্দ্রনাথ'], ['কিন্তু', 'কী', 'ত্যাগ', 'কিসেরই', 'বা', 'তপস্যা', 'রবীন্দ্রনাথ', 'তাকে', 'বলছেন', 'যেখানে', 'আমিকে', 'নাআমির', 'দিকে', 'ছাড়তে', 'বাধা', 'পাই', 'তাকে', 'অহং', 'বেড়ায়', 'বিচ্ছিন্ন', 'সীমাবদ্ধ', 'করে', 'দেখি'], ['এক', 'আত্মলোকে', 'সকল', 'আত্মর', 'অভিমুখে', 'আত্মার', 'সত্য', 'এই', 'সত্যের', 'আদর্শেই', 'বিচার', 'করতে', 'হবে', 'মানুষের', 'সভ্যতা'], ['মানুষের', 'দায়', 'মহামানবের', 'দায়', 'কোথাও', 'তার', 'সীমা', 'নেই'], ['রবীন্দ্রনাথ', 'যাকে', 'বলছেন', 'মহামানবের', 'দায়', 'তা', 'হল', 'আমি', 'থেকে', 'নাআমিতে', 'পৌঁছনো'], ['আবরণময়', 'আমিকে', 'আবরণমুক্ত', 'আত্মতে', 'উত্তরণ'], ['সেটাই', 'তাঁর', 'সাধনা'], ['রবীন্দ্রনাথ', 'যাকে', 'বলেন', 'হওয়া', 'বা', 'হয়ে', 'ওঠা'], ['এই', 'হয়ে', 'ওঠার', 'সাধনাই', 'রবীন্দ্রনাথের', 'জীবনজুড়ে', 'তাঁর', 'সমগ্র', 'রচনার', 'ধারণশক্তি']]


Now that we have our preprocessed training data, we can start training our model.

We will generate embeddings for each word of size 200 and use 5 words in its vicinity to figure out the meaning of the word

In [10]:
model = Word2Vec(body, size=200, window=5, min_count=1)

In [21]:
print("What are the words most similar to chele")
model.wv.most_similar('ছেলে', topn=5)

What are the words most similar to chele


[('মেয়ে', 0.9279561042785645),
 ('ভাই', 0.8839391469955444),
 ('বোন', 0.8741269111633301),
 ('বাবা', 0.8684152364730835),
 ('বন্ধু', 0.8629558682441711)]

In [20]:
print("What is Father + Girl - Boy =?")
model.wv.most_similar(positive=['বাবা', 'মেয়ে'], negative=['ছেলে'], topn=5)

What is Father + Girl - Boy =?


[('মা', 0.910052478313446),
 ('বাবামা', 0.8313219547271729),
 ('স্ত্রী', 0.7864214777946472),
 ('সন্তান', 0.7833602428436279),
 ('বন্ধুরা', 0.7827222347259521)]

In [22]:
print('Find the odd one out')
model.wv.doesnt_match("কলকাতা চেন্নাই দিল্লি রবীন্দ্রনাথ".split())

Find the odd one out


'রবীন্দ্রনাথ'

In [23]:
print("How similar are bengali and sweet?")
model.wv.similarity('বাঙালি', 'মিষ্টি')

How similar are bengali and sweet?


0.6867578

In [25]:
model.wv.save_word2vec_format('news_vector_text.txt', binary=False)
model.wv.save_word2vec_format('news_vector_binary.txt', binary=True)

In [26]:
print("What about Bihari and Sweets?")
model.wv.similarity('বিহারি', 'মিষ্টি')

What about Bihari and Sweets?


0.5881788